In [ ]:
# !pip install q tensorflow==2.6.0# install tensorflow

In [ ]:
!pip install keras-metrics

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 133 kB 19.1 MB/s 


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow import keras 

import keras_tuner as kt
from kerastuner.tuners import BayesianOptimization
from tensorflow.keras.utils import plot_model
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from tensorflow.keras.layers import Bidirectional



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  if __name__ == '__main__':


In [ ]:
import pandas as pd
from sklearn import preprocessing
from numpy import array
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from keras.utils import np_utils
from keras.layers.merge import concatenate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Lodaing Data
df=pd.read_csv("/content/drive/MyDrive/Paper2-LSTM-TS/Data-Timestep_modality/time_step3.csv")
df

,PATNO,EVENT_ID,NUPSOURC1a,NP1COG,NP1HALL,NP1ANXS,NP1APAT,NP1DDS,NUPSOURC1,NP1SLPN,...,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,NHY_y1,DEPRS_y2,DEPRS_scale_y3
0,3001,BL,1,0,0,1,0,0,1,0,...,3,0,0,2,0,0,0,1,0,0
1,3001,V04,1,1,0,1,1,0,1,0,...,1,0,0,2,0,0,0,2,1,1
2,3001,V06,1,0,0,0,0,0,1,1,...,2,1,1,2,0,0,0,2,0,0
3,3002,BL,1,1,0,1,0,0,1,0,...,1,3,3,1,0,3,0,2,0,1
4,3002,V04,1,1,0,1,0,0,1,0,...,1,2,2,3,0,2,1,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3172,92490,V04,1,1,0,0,2,0,1,0,...,1,1,0,1,0,1,0,1,0,0
3173,92490,V06,1,0,0,0,0,0,1,2,...,1,1,1,2,0,1,0,1,0,0
3174,92834,BL,1,0,0,0,0,0,1,4,...,1,1,1,2,0,0,0,0,0,0
3175,92834,V04,1,0,0,0,0,0,1,3,...,1,1,1,3,0,1,0,0,0,0


In [ ]:
# dropping the patient ID
df = df.drop(['PATNO', 'EVENT_ID','NHY_y1','DEPRS_scale_y3'], axis =1, inplace = False);
df.head()

,NUPSOURC1a,NP1COG,NP1HALL,NP1ANXS,NP1APAT,NP1DDS,NUPSOURC1,NP1SLPN,NP1SLPD,NP1PAIN,...,PTCGBOTH_y,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,DEPRS_y2
0,1,0,0,1,0,0,1,0,1,0,...,1,1,3,0,0,2,0,0,0,0
1,1,1,0,1,1,0,1,0,2,0,...,1,0,1,0,0,2,0,0,0,1
2,1,0,0,0,0,0,1,1,2,1,...,1,1,2,1,1,2,0,0,0,0
3,1,1,0,1,0,0,1,0,1,1,...,1,3,1,3,3,1,0,3,0,0
4,1,1,0,1,0,0,1,0,2,0,...,1,3,1,2,2,3,0,2,1,1


In [ ]:
df.tail()

,NUPSOURC1a,NP1COG,NP1HALL,NP1ANXS,NP1APAT,NP1DDS,NUPSOURC1,NP1SLPN,NP1SLPD,NP1PAIN,...,PTCGBOTH_y,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,DEPRS_y2
3172,1,1,0,0,2,0,1,0,2,1,...,1,1,1,1,0,1,0,1,0,0
3173,1,0,0,0,0,0,1,2,2,1,...,1,1,1,1,1,2,0,1,0,0
3174,1,0,0,0,0,0,1,4,2,2,...,1,3,1,1,1,2,0,0,0,0
3175,1,0,0,0,0,0,1,3,2,4,...,1,2,1,1,1,3,0,1,0,0
3176,1,0,0,0,0,0,1,3,2,4,...,1,1,1,1,1,2,0,1,0,0


In [ ]:
#data 
steps=3

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps=3):
	X, y = list(), list()
	for i in range(0,len(sequences),n_steps):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 

def normalization(x):
  # x = df.iloc[:,:-2].values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
    x_norm = min_max_scaler.fit_transform(x)
    return x_norm

from sklearn.preprocessing import OneHotEncoder



def encode_Y(Y_raw):
    #Y_raw = Y_raw.astype('int32')
    # encode class values as integers
    encoder = LabelEncoder()
    encoder.fit(Y_raw)
    encoded_Y = encoder.transform(Y_raw)
    # convert integers to dummy variables (i.e. one hot encoded)
    encoded_y = np_utils.to_categorical(encoded_Y)
    return encoded_y


def splitY (Y_Vector):
    Y0 = encode_Y(Y_Vector[:,0])
    # Y1 = encode_Y(Y_Vector[:,1])
       
    return Y0 #,Y1   



In [ ]:
dx, dy = split_sequences(df.values)

In [ ]:
dx

array([[[1, 0, 0, ..., 0, 0, 0],
        [1, 1, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]],

       [[1, 1, 0, ..., 0, 3, 0],
        [1, 1, 0, ..., 0, 2, 1],
        [1, 2, 0, ..., 0, 1, 0]],

       [[1, 0, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]],

       ...,

       [[1, 0, 0, ..., 0, 3, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 2, 0]],

       [[1, 0, 0, ..., 0, 1, 0],
        [1, 1, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0]],

       [[1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 1, 0],
        [1, 0, 0, ..., 0, 1, 0]]])

In [ ]:
dy

array([[0],
       [1],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [ ]:
y = encode_Y(dy[:,0])
splitY(dy);


In [ ]:
# Modal

# Model configuration
batch_size = 10 #50
loss_function = sparse_categorical_crossentropy
no_classes = 2 #n
no_epochs = 5  # 50
validation_split = 0.2
verbosity = 1

In [ ]:
#biLSTM
def build_model(hp):

  model = Sequential()
  model.add(Bidirectional(LSTM(hp.Int('input_unit',min_value=32,max_value=512,step=32),return_sequences=True, input_shape=(30, 145))))
  
  for i in range(hp.Int('n_layers', 1, 4)):
      model.add(Bidirectional(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=32),return_sequences=True)))
      model.add(Bidirectional(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=32))))
      model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
      model.add(Dense(trainY.shape[0], activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
      model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse'])

    # # Display the model summary
    #   model.summary()

    # Compile the model
      model.compile(loss=loss_function,
                    optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                    metrics=['accuracy'])
    
    # Return the model
      return model

# define model
tuner = BayesianOptimization(
        build_model,
        objective='accuracy',
        max_trials=5 , # 5 (is good ),7, 10
        executions_per_trial=5, # 2,3,5,
        #directory=os.path.normpath('C:/keras_tuning'),
        project_name='timeseries_bayes_opt_POC',
        overwrite=True)

In [ ]:
# Display search space summary

tuner.search_space_summary()

In [ ]:
# Perform random search
tuner.search(trainX, trainY, epochs=no_epochs, validation_split=validation_split)

In [ ]:
# Get best model
models = tuner.get_best_models(num_models=1)
best_model = models[0]

In [ ]:
# Fit data to model
history = best_model.fit(trainX, trainY,
            batch_size=batch_size,
            epochs=no_epochs,
            verbose=verbosity,
            validation_split=validation_split)

In [ ]:
# Generate generalization metrics
score = best_model.evaluate(testX,testY, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


In [ ]:
plot_model(best_model, to_file='model.png')

In [ ]:
tuner.results_summary()